In [85]:
import os, sys
sys.path.append('../')

import outlines
import outlines.models as models
import outlines.text as text
from outlines.text.generate.sample import multinomial

import torch
import transformers

from pydantic import BaseModel, Field, constr, conlist
from enum import Enum

from utils.summarize_utils import ConstrainedResponseHST, prompt_fn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# model = models.awq("TheBloke/Mistral-7B-OpenOrca-AWQ")
model = models.awq("TheBloke/OpenHermes-2.5-Mistral-7B-AWQ")

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Replacing layers...: 100%|█| 32/32 [00:02<00:00, 13.59it/s


  0%|          | 0/739 [00:00<?, ?w/s]

Fusing layers...: 100%|███| 32/32 [00:04<00:00,  6.46it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [70]:
# model_name = "mosaicml/mpt-7b-8k-instruct"
# model_name = "teknium/OpenHermes-2.5-Mistral-7B"

# config = transformers.AutoConfig.from_pretrained(
#     model_name, trusmt_remote_code=True
# )
# config.init_device = "meta"
# model = models.transformers(
    
#     model_name=model_name",
#     device="cuda",
#     model_kwargs={
#         "config": config,
#         "trust_remote_code": True,
#         "torch_dtype": torch.bfloat16,
#         "device_map": {"": 0},
#         "cache_dir": "/n/holystore01/LABS/iaifi_lab/Users/smsharma/hf_cache/"
#     },
# )

In [103]:
abstract = """
We propose to observe ultraluminous X-ray sources which are located near
objects bright both in the X-rays and the optical using Chandra and HST. The
presence of these reference objects will allow us to tie the x-ray and optical
references frames and achieve 0.1-0.2 arcsecond relative position accuracy in
searching for optical counterparts to the ultraluminous x-ray sources. This
will be a significant improvement over the accuracy previously obtained for
most ULXs {limited by Chandra's absolute astrometry} and will should permit
identification of individual counterparts.
"""

# abstract = """
# We propose a comprehensive optical, UV, and X-ray investigation of the unique
# galaxy POX 52. POX 52 is a Seyfert 1 galaxy with unprecedented properties: its
# host galaxy appears to be a dwarf elliptical, and its stellar velocity
# dispersion is only 36 km/s. The stellar velocity dispersion and the broad
# emission-line widths both suggest a black hole mass of order 10^5 solar
# masses, placing POX 52 in a region of AGN parameter space that is almost
# completely unexplored at present. We request ACS/HRC imaging to perform a
# definitive measurement of the host galaxy structure; STIS UV and optical
# spectroscopy to study the nonstellar continuum and the structure of the
# broad-line region; and Chandra ACS imaging to detect the X-ray emission from
# the nucleus and investigate its spectral and variability properties. The
# results of this program will give a detailed understanding of the host galaxy
# and accretion properties of one of the very few known black holes in the mass
# range around 10^5 solar masses.
# """

# abstract = """
# The observed optical depths to microlensing of stars in the Galactic bulge are
# difficult to reconcile with our present understanding of Galactic dynamics.
# The main source of uncertainty in those comparisons is now shifting from
# microlensing measurements to the dynamical models of the Galactic bar. We
# propose to constrain the Galactic bar models with proper motion observations
# of Bulge stars that underwent microlensing by determining both the kinematic
# identity of the microlensed sources and the importance of streaming motions.
# The lensed stars are typically farther than randomly selected stars.
# Therefore, our proper motion determinations for 36 targeted MACHO events will
# provide valuable constraints on the dynamics of bulge stars as a function of
# distance. The first epoch data for our proposed events is already available in
# the HST archive so the project can be completed within a single HST cycle. The
# exceptional spatial resolution of HST is essential for completion of the
# project. Constraints on te total mass in the bulge will ultimately lead to
# the determination of the amount of dark matter in inner Galaxy.
# """

In [104]:
prompt = prompt_fn(abstract)
generator = text.generate.json(model, ConstrainedResponseHST, sampler=multinomial)
sequence = generator(prompt)
sequence

ConstrainedResponseHST(objects_and_phenomena="ultraluminous X-ray sources, X-ray and optical bright objects, reference frames, optical counterparts, Chandra's absolute astrometry", science_use_cases='tie X-ray and optical references frames, achieve 0.1-0.2 arcsecond relative position accuracy, identification of individual counterparts')

In [73]:
# generator = text.generate.continuation(model)
# sequence = generator(prompt)
# sequence

In [74]:
from utils.abstract_utils import read_abstracts_file

from tqdm.notebook import tqdm

filename = "../data/abstracts.cat"

abstracts_df = read_abstracts_file(filename)

# Drop rows with missing Cycle
abstracts_df = abstracts_df.dropna(subset=['Cycle'])
abstracts_df = abstracts_df[abstracts_df['Cycle'] != '']

# Convert Cycle and ID to int
abstracts_df['Cycle'] = abstracts_df['Cycle'].astype(int)
abstracts_df['ID'] = abstracts_df['ID'].astype(int)

In [76]:
# import pandas as pd

# data_folder = "../data/observations_v1/"

# def get_abstracts(data_folder, max_abstracts=10):
    
#     # Lists to store results
#     proposal_id_list = []
#     objects_list = []
#     science_list = []

#     # Collect directories that contain .jpg files and match the "proposal_" pattern, excluding unwanted directories
#     directories_with_images = [os.path.join(r, d)
#                                for r, dirs, files in os.walk(data_folder)
#                                for d in dirs
#                                if d.startswith("proposal_") and not d.endswith('.ipynb_checkpoints')]

#     # Walk through data folder
#     for directory in tqdm(directories_with_images[:max_abstracts]):
#         proposal_id = directory.split("proposal_")[-1]  # Extract proposal id from the directory name

#         # Extract abstract using the dataframe
#         abstract = abstracts_df[abstracts_df["ID"] == int(proposal_id)]["Abstract"].values[0]

#         prompt = prompt_fn(abstract)
#         generator = text.generate.json(model, ConstrainedResponseHST)
#         result = generator(prompt)
#         print(result)
#         print("\n")

#         proposal_id_list.append(proposal_id)
#         science_list.append(result.science_use_cases)
#         objects_list.append(result.objects_and_phenomena)

#     return proposal_id_list, objects_list, science_list
        
# proposal_id_list, objects_list, science_list = get_abstracts(data_folder)

# # Create a DataFrame
# df = pd.DataFrame({
#     'proposal_id': proposal_id_list,
#     'objects_phenomena': objects_list,
#     'science_use_cases': science_list
# })

# df.to_csv('../data/summary_v1.csv', index=False)